# Comparativo nacional con foco en la Región de Los Ríos

Analizamos la oferta formativa 2025 considerando todas las regiones del país, resaltando la Región de Los Ríos como eje principal. Las visualizaciones combinan gráficos de barras y líneas con Plotly para facilitar insights interactivos sobre vacantes, costos y cantidad de programas.

In [1]:
from pathlib import Path
import unicodedata

import numpy as np
import pandas as pd
import plotly.express as px

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_PATH = PROJECT_ROOT / "data/processed/oferta_formativa_clean.parquet"
if not DATA_PATH.exists():
    raise FileNotFoundError(f"No se encontró el archivo esperado: {DATA_PATH}")

df = pd.read_parquet(DATA_PATH)
df.shape

(3328, 57)

In [2]:
def normalize(value: str) -> str:
    if not isinstance(value, str):
        return ""
    normalized = unicodedata.normalize("NFKD", value)
    return ''.join(ch for ch in normalized if not unicodedata.combining(ch)).lower()

focus_mask = df['region_sede'].apply(normalize).str.contains('los rios', na=False)
focus_region = df.loc[focus_mask, 'region_sede'].mode().iloc[0] if focus_mask.any() else 'Los Ríos'

regions_summary = (
    df.groupby('region_sede', as_index=False)
    .agg(
        vacantes_totales=('total_vacantes', 'sum'),
        programas_unicos=('codigo_carrera', 'nunique'),
        arancel_promedio=('arancel_anual', 'mean'),
        matricula_promedio=('matricula_anual', 'mean')
    )
)
regions_summary['region_categoria'] = np.where(
    regions_summary['region_sede'] == focus_region,
    'Región de Los Ríos',
    'Otras regiones'
)
color_map = {
    'Región de Los Ríos': '#d62728',
    'Otras regiones': '#9ecae1'
}
regions_summary.sort_values('vacantes_totales', ascending=False, inplace=True)
regions_summary.head()


,region_sede,vacantes_totales,programas_unicos,arancel_promedio,matricula_promedio,region_categoria
0,Biobío,54004,579,3354541.863578,209711.363987,Otras regiones
1,La Araucanía,49055,468,3115429.561358,209868.547341,Otras regiones
2,Los Lagos,18821,244,2987438.696183,218639.824427,Otras regiones
3,Los Ríos,9877,236,3604608.616715,200591.930836,Región de Los Ríos


## Vacantes totales por región
La barra compara la oferta agregada de vacantes. Resaltamos Los Ríos para dimensionar su aporte relativo.

In [3]:
fig_vacantes = px.bar(
    regions_summary,
    x='vacantes_totales',
    y='region_sede',
    color='region_categoria',
    color_discrete_map=color_map,
    text='vacantes_totales',
    orientation='h',
    title='Vacantes totales por región (destacando Los Ríos)',
    labels={'region_sede': 'Región', 'vacantes_totales': 'Vacantes'}
)
fig_vacantes.update_layout(yaxis={'categoryorder': 'total ascending'}, height=750)
fig_vacantes

## Línea de arancel promedio anual
Ordenamos las regiones por arancel promedio para observar diferencias de costo. La serie marca explícitamente la Región de Los Ríos.

In [4]:
ordered = regions_summary.sort_values('arancel_promedio')
fig_arancel = px.line(
    ordered,
    x='region_sede',
    y='arancel_promedio',
    markers=True,
    color='region_categoria',
    color_discrete_map=color_map,
    title='Arancel promedio anual por región',
    labels={'region_sede': 'Región', 'arancel_promedio': 'Arancel promedio (CLP)'}
)
fig_arancel.update_traces(line_width=3)
fig_arancel.update_layout(xaxis_tickangle=-45, height=600)
fig_arancel

## Programas únicos por región
Otra perspectiva de escala es el número de programas diferentes que oferta cada región.

In [5]:
fig_programas = px.bar(
    regions_summary.sort_values('programas_unicos', ascending=False),
    x='region_sede',
    y='programas_unicos',
    color='region_categoria',
    color_discrete_map=color_map,
    title='Programas únicos por región',
    labels={'region_sede': 'Región', 'programas_unicos': 'Programas únicos'}
)
fig_programas.update_layout(xaxis_tickangle=-45, height=600)
fig_programas

## Línea de matrícula anual promedio
La matrícula promedio entrega contexto sobre los costos iniciales que enfrentan las y los postulantes.

In [6]:
ordered_matricula = regions_summary.sort_values('matricula_promedio')
fig_matricula = px.line(
    ordered_matricula,
    x='region_sede',
    y='matricula_promedio',
    markers=True,
    color='region_categoria',
    color_discrete_map=color_map,
    title='Matrícula anual promedio por región',
    labels={'region_sede': 'Región', 'matricula_promedio': 'Matrícula promedio (CLP)'}
)
fig_matricula.update_traces(line_width=3)
fig_matricula.update_layout(xaxis_tickangle=-45, height=600)
fig_matricula